## Data Exploration

##### Goals:
- To see the nature of the datasets i.e. their columns, null values, number of records, etc.
- To see overall statistics — no. of tv shows, no. of movies, types of movie categories, etc.

#### Importing Libraries


In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

#### Loading CSV files into SQLite Database

In [ ]:
# Loading as pandas tables first
netflix = pd.read_csv("data/netflix_titles.csv")
imdb_mov = pd.read_csv("data/IMDb movies.csv")
imdb_rat = pd.read_csv("data/IMDb ratings.csv")

# loading the pandas tables into netflix_imdb database as three separate tables
conn = sqlite3.connect('data/netflix_imdb.db')
cursor = conn.cursor()
netflix.to_sql("netflix", conn, if_exists="replace", index=False)
imdb_mov.to_sql("imdb_m", conn, if_exists="replace", index=False)
imdb_rat.to_sql("imdb_r", conn, if_exists="replace", index=False)

#### Previewing data
Notes:
- Some records have director = None

In [ ]:
print('='*60)
print('NETFLIX DATA PREVIEW')
print('='*60)
cursor.execute("SELECT * FROM netflix LIMIT 5;") # SQL query to preview
pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description]).head()

In [ ]:
print('='*60)
print('IMDB MOVIES DATA PREVIEW')
print('='*60)
cursor.execute("SELECT * FROM imdb_m LIMIT 5;") # SQL query to preview
pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description]).head()

In [ ]:
print('='*60)
print('IMDB RATINGS DATA PREVIEW')
print('='*60)
cursor.execute("SELECT * FROM imdb_r LIMIT 5;") # SQL query to preview
pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description]).head()

#### Column Exploration:

Comments:
- There are two title columns in IMDb movies table which have to be reconciled to connect with        netflix table
- The imdb_title_id will be used to connect the imdb movies and imdb ratings table
- Countries, cast, and listed_in columns in netflix table has multiple countries separated by commas.

In [ ]:
print('='*60)
print("NETFLIX DATASET COLUMNS AND TYPES")
print("(column_index, name, data type, not null, default value, is primary key)")
print('='*60)
cursor.execute("PRAGMA table_info(netflix);")
for col in cursor.fetchall():
    print(col)

print('='*60)
print("IMDB MOVIES DATASET COLUMNS AND TYPES")
print("(column_index, name, data type, not null, default value, is primary key)")
print('='*60)
cursor.execute("PRAGMA table_info(imdb_m);")
for col in cursor.fetchall():
    print(col)

print('='*60)
print("IMDB RATINGS DATASET COLUMNS AND TYPES")
print("(column_index, name, data type, not null, default value, is primary key)")
print('='*60)
cursor.execute("PRAGMA table_info(imdb_r);")
for col in cursor.fetchall():
    print(col)

##### Found: In imdb movies dataset, title not equal to original title when title is translated in English

In [ ]:
# Since there are two title columns in imdb movies table (imdb_m), let's check discrepencies:
cursor.execute("SELECT title, original_title FROM imdb_m where title is not original_title") # SQL query

# converted to a dataframe for better formatting
pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description]).head()

##### Found: Some netflix and imdb titles are repeated with different imdb_ratings

Example: title = Paranoid

In the IMDb dataset specifically, there's nuance:
- Many films have the same titles, but are created by different direcors.
- Some have same titles and same directors, but they are released in different years, possibly remakes.
- Even then, there are some which are released in the same year, but their durations and languages are different.

So in this way, **all entries in IMDb dataset are unique**

In [ ]:
# Joining the two datasets on title alone (note: not good as titles are repeated,
#                                           need to find addtional join parameters)
cursor.execute("""
               SELECT
                    n.title  AS netflix_title,
                    i.title AS imdb_title,
                    i.avg_vote AS imdb_rating
                FROM netflix n
                JOIN imdb_m i
                ON n.title = i.title;
                """) # SQL JOIN query to see IMDb ratings of movies and tvshows on netflix.

# # investigating the matter
cursor.execute("""
               SELECT title, year, director, duration,language
                FROM imdb_m
                WHERE (title, director, year) IN (
                    SELECT title, director, year
                    FROM imdb_m
                    GROUP BY title, director, year
                    HAVING COUNT(*) > 1
                )
                ORDER BY title;
                """)
pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

#### Category statistics:

- There are only 8,807 netflix titles but 86,855 titles registered in IMDb datasets

In [ ]:
cursor.execute("SELECT COUNT(*) FROM netflix;") # Querying total listing counts
total_listings = cursor.fetchall()[0][0]

############# Content Type Pie Chart VISUALIZATION #############
cursor.execute("SELECT type, COUNT(*) as count FROM netflix GROUP BY type") # Querying movie counts
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df.set_index('type').plot.pie(y='count', autopct='%1.1f%%', figsize=(7,7), legend=False, colors=['skyblue','lightgreen'])
plt.ylabel('')
plt.title(f"Netflix Content by Type (Total = {total_listings})")
plt.show()
#########################################


############# Age Ratings Pie Chart VISUALIZATION #############
df = pd.read_sql_query("SELECT rating FROM netflix where rating IS NOT NULL", conn)
rating_counts = df['rating'].value_counts()

# Compute percentage of each rating
total = rating_counts.sum()
rating_pct = rating_counts / total

# Group ratings with <3% into 'Other'
rating_counts_grouped = rating_counts.copy()
rating_counts_grouped[rating_pct < 0.03] = 0
other_sum = rating_counts[rating_pct < 0.03].sum()

# Drop zero rows and add 'Other'
rating_counts_grouped = rating_counts_grouped[rating_counts_grouped > 0]
rating_counts_grouped['Other'] = other_sum

plt.figure(figsize=(7,7))
plt.pie(
    rating_counts_grouped,
    labels=rating_counts_grouped.index,
    autopct='%1.1f%%',
    startangle=40,
    colors=plt.cm.Pastel1.colors,
    labeldistance=0.8
)
plt.title("Netflix Content Distribution by Rating (Grouped <3% as Other)")
plt.axis('equal')  # keeps pie circular
plt.show()
#########################################
#################################################################

print("="*60)
cursor.execute("SELECT COUNT(*) FROM imdb_m;") # SQL SELECT query
total_imdb_listings = cursor.fetchall()[0][0]



#### Release Year Histograms for IMDb listings

Seeing how many movies released per year and the average IMDb rating across time

In [ ]:
query = """
SELECT
    (CAST(year AS INTEGER) / 10) * 10 AS decade,
    COUNT(*) AS count
FROM imdb_m
WHERE CAST(year AS INTEGER) > 0
GROUP BY decade
ORDER BY decade ASC;
"""

df = pd.read_sql(query, conn) # total counts df

query = """
    WITH ranked AS (
        SELECT
            *,
            ((CAST(year AS INTEGER) / 10) * 10) AS decade,
            ROW_NUMBER() OVER (
                PARTITION BY ((CAST(year AS INTEGER) / 10) * 10)
                ORDER BY avg_vote DESC
            ) AS rn
        FROM imdb_m
        WHERE CAST(year AS INTEGER) > 0 AND avg_vote IS NOT NULL
        )
    SELECT
        decade,
        COUNT(*) AS count,
        AVG(avg_vote) AS top10_avg_vote
    FROM ranked
    WHERE rn <= 10  -- only top 10 per decade
    GROUP BY decade
    ORDER BY decade ASC;
"""
df_top_10_avg_ratings = pd.read_sql(query, conn)

# Plot
df_plot = df.merge(df_top_10_avg_ratings[['decade', 'top10_avg_vote']], on='decade', how='left')

fig, ax1 = plt.subplots(figsize=(10,5))

# Left y-axis: total count
ax1.plot(df_plot['decade'], df_plot['count'], color='tab:blue', marker='o', label='Total IMDb listings')
ax1.set_xlabel("Decade")
ax1.set_ylabel("Number of IMDb listings", color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Right y-axis: top 10 average rating
ax2 = ax1.twinx()
ax2.plot(df_plot['decade'], df_plot['top10_avg_vote'], color='tab:red', marker='o', label='Top 10 avg rating')
ax2.set_ylabel("Top 10 IMDb avg rating", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Optional: combined legend
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

# Title and grid
plt.title("Total IMDb Listings and Average of Top 10 IMDb Ratings per Decade")
ax1.grid(True)
plt.show()
print("="*30,"Comments","="*30)
print("""
      - The total number of IMDb listings have increased over the decades as more movies have been made.
      - The drop in 2020 is because the dataset ends at 2020 and the datapoint was supposed to aggregate
      movie counts from 2020-2025
      - Plotting the average of top 10 IMDb ratings per decade points that the roughly the quality of movies is increasing.
      """)

#### Countries Statistics

In [ ]:
print("="*20, "TOP 10 COUNTRIES WHERE NETFLIX LISTINGS ARE AVAILABLE OR MADE","="*20)


query = """
        SELECT country, COUNT(*) AS count  FROM netflix
        WHERE country IS NOT NULL
        GROUP BY country
        ORDER BY count DESC
"""
df = pd.read_sql_query(query, conn) # some records in df have mulltiple countries in the country column

# So we have to get all rows and then write custom logic to get the counts in python and then sort.
cursor.execute("""
                SELECT country
                FROM netflix
                WHERE country IS NOT NULL
                """)

from collections import Counter
country_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (countries,) in cursor: # countries = 'Argentina, France' for example.
    for country in countries.split(','): # country = 'Argentina' and then country = ' France'
        country_counter[country.strip()] += 1

country_df = (
    pd.DataFrame(country_counter.items(), columns=['country', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)

################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(country_df['country'], country_df['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 Countries by Netflix Listings")
plt.gca().invert_yaxis()  # largest at top
plt.show()
#################################################

#### NETFLIX Director Statistics

In [ ]:
print("="*20, "TOP 10 DIRECTORS OF NETFLIX LISTINGS","="*20)

# some records in df have mulltiple directors in the director column

# So we have to get all rows and then write custom logic to get the counts in python and then sort.
cursor.execute("""
                SELECT director
                FROM netflix
                WHERE director IS NOT NULL and type = 'Movie'
                """)

from collections import Counter
director_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (directors,) in cursor:
    for director in directors.split(','):
        director_counter[director.strip()] += 1

director_df_movies = (
    pd.DataFrame(director_counter.items(), columns=['director', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)

################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(director_df_movies['director'], director_df_movies['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 Directors of Movies on Netflix")
plt.gca().invert_yaxis()  # largest at top
plt.show()
#################################################

In [ ]:
cursor.execute("""
                SELECT director
                FROM netflix
                WHERE director IS NOT NULL and type = 'TV Show'
                """) # getting TV show directors

from collections import Counter
director_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (directors,) in cursor:
    for director in directors.split(','):
        director_counter[director.strip()] += 1

director_df_shows = (
    pd.DataFrame(director_counter.items(), columns=['director', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)
################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(director_df_shows['director'], director_df_shows['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 Directors of TV Shows on Netflix")
plt.gca().invert_yaxis()  # largest at top
plt.show()
#################################################

#### IMDb director statistics

In [ ]:
print("="*20, "TOP 10 DIRECTORS OF IMDb LISTINGS","="*20)

# some records in df have mulltiple directors in the director column

# So we have to get all rows and then write custom logic to get the counts in python and then sort.
cursor.execute("""
                SELECT director
                FROM imdb_m
                WHERE director IS NOT NULL
                """)

from collections import Counter
director_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (directors,) in cursor:
    for director in directors.split(','):
        director_counter[director.strip()] += 1

director_df_imdb = (
    pd.DataFrame(director_counter.items(), columns=['director', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)

################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(director_df_imdb['director'], director_df_imdb['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 Directors of listings on IMDb")
plt.gca().invert_yaxis()  # largest at top
plt.show()
#################################################
director_df_imdb

#### IMDb Top 10 Actors of all time

Comments:
- The analysis correctly suggests that Brahmanandam (Indian Comedy Drama Actor) has the most number of movies.
- In reality he has a Guiness World Record for over 1000 movies, but IMDb has only these many.

In [ ]:
print("="*20, "TOP 10 ACTORS OF IMDb LISTINGS","="*20)

# some records in df have mulltiple directors in the director column

# So we have to get all rows and then write custom logic to get the counts in python and then sort.
cursor.execute("""
                SELECT actors
                FROM imdb_m
                WHERE actors IS NOT NULL
                """)

from collections import Counter
actor_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (actors,) in cursor:
    for actor in actors.split(','):
        actor_counter[actor.strip()] += 1

actor_df = (
    pd.DataFrame(actor_counter.items(), columns=['actor', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)

################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(actor_df['actor'], actor_df['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 Actors of listings on IMDb")
plt.gca().invert_yaxis()  # largest at top
plt.show()
#################################################
actor_df

#### IMDb Listing Genres

In [ ]:
print("="*20, "TOP 10 Genres OF IMDb LISTINGS","="*20)

# some records in df have mulltiple directors in the director column

# So we have to get all rows and then write custom logic to get the counts in python and then sort.
cursor.execute("""
                SELECT genre
                FROM imdb_m
                WHERE genre IS NOT NULL
                """)

from collections import Counter
genre_counter = Counter() # a good built in functionality for countign using a dictionary: default value = 0.

for (genre,) in cursor:
    for genre in genre.split(','):
        genre_counter[genre.strip()] += 1

genre_df = (
    pd.DataFrame(genre_counter.items(), columns=['genre', 'count'])
      .sort_values(by='count', ascending=False)
      .head(10)
)

################### VISUALIZATION ##############
plt.figure(figsize=(9,5))
plt.barh(genre_df['genre'], genre_df['count'])
plt.xlabel("Number of Titles")
plt.title("Top 10 genres of listings on IMDb")
plt.gca().invert_yaxis()  # largest at top
plt.show()

print("""
        COMMENTS:
        - These percentages are from the total movies in top 10 genres and not all.
        - Plus since a movie can have multiple genres, it simply shows a visual of dominating genres as they overlap.
      """)
genre_df.set_index('genre').plot.pie(y='count', autopct='%1.1f%%', figsize=(7,7), legend=False)
plt.ylabel('')
plt.title(f"IMDb Genres")
plt.show()
#################################################